In [1]:
!pip install -U keras-tuner

     |████████████████████████████████| 61kB 3.3MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=7d1a79c3d192de3d8c69da307c2d504fb844547e15aef87dcbf993c57c49f755
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=7efb3c68ea0c998dcfe7eaa4e0a75f886f9da64466e1cde544a402c8c2abf16e
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

In [3]:
tf.__version__

'2.3.0'

In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [6]:
## Scaling the images because images are in greyscale
train_images = train_images / 255.0
test_images = test_images / 255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [9]:
train_images.shape

(60000, 28, 28, 1)

In [10]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters = hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
          kernel_size = hp.Choice('conv_1_kernel', values=[3, 5]),
          activation = 'relu',
          input_shape = (28, 28, 1)
      ),
      keras.layers.Conv2D(
          filters = hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
          kernel_size = hp.Choice('conv_2_kernel', values=[3, 5]),
          activation = 'relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units = hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation = 'relu'
      ),
      keras.layers.Dense(10, activation='softmax')                      
  ])

  model.compile(
      optimizer = keras.optimizers.Adam(
          hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
          loss = 'sparse_categorical_crossentropy',
          metrics = ['accuracy']
      )
  
  return model

In [11]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [12]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name='Mnist Fashion')

In [14]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.3829 - accuracy: 0.8615 - val_loss: 0.2963 - val_accuracy: 0.8895
Epoch 2/3
1688/1688 [==============================] - 12s 7ms/step - loss: 0.2467 - accuracy: 0.9085 - val_loss: 0.2491 - val_accuracy: 0.9087
Epoch 3/3
1688/1688 [==============================] - 11s 7ms/step - loss: 0.1874 - accuracy: 0.9303 - val_loss: 0.2498 - val_accuracy: 0.9100


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.5302 - accuracy: 0.8128 - val_loss: 0.4094 - val_accuracy: 0.8568
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3672 - accuracy: 0.8709 - val_loss: 0.3427 - val_accuracy: 0.8710
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3143 - accuracy: 0.8869 - val_loss: 0.3144 - val_accuracy: 0.8848


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.6062 - accuracy: 0.7953 - val_loss: 0.4236 - val_accuracy: 0.8495
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3970 - accuracy: 0.8615 - val_loss: 0.3745 - val_accuracy: 0.8642
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3459 - accuracy: 0.8787 - val_loss: 0.3435 - val_accuracy: 0.8747


Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4135 - accuracy: 0.8502 - val_loss: 0.3222 - val_accuracy: 0.8840
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2741 - accuracy: 0.8997 - val_loss: 0.2837 - val_accuracy: 0.8947
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2235 - accuracy: 0.9169 - val_loss: 0.2714 - val_accuracy: 0.9020


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4629 - accuracy: 0.8336 - val_loss: 0.3884 - val_accuracy: 0.8510
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3400 - accuracy: 0.8732 - val_loss: 0.3294 - val_accuracy: 0.8760
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3052 - accuracy: 0.8863 - val_loss: 0.3363 - val_accuracy: 0.8818


INFO:tensorflow:Oracle triggered exit


In [15]:
model = tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 96)        2496      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 128)       110720    
_________________________________________________________________
flatten (Flatten)            (None, 61952)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               6938736   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 7,053,082
Trainable params: 7,053,082
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0271 - accuracy: 0.9907 - val_loss: 0.0917 - val_accuracy: 0.9797
Epoch 5/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0227 - accuracy: 0.9922 - val_loss: 0.0962 - val_accuracy: 0.9805
Epoch 6/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0180 - accuracy: 0.9940 - val_loss: 0.0990 - val_accuracy: 0.9778
Epoch 7/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0191 - accuracy: 0.9936 - val_loss: 0.1252 - val_accuracy: 0.9717
Epoch 8/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0195 - accuracy: 0.9931 - val_loss: 0.1371 - val_accuracy: 0.9735
Epoch 9/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0182 - accuracy: 0.9943 - val_loss: 0.1576 - val_accuracy: 0.9690
Epoch 10/10
1688/1688 [==============================] - 11s 7ms/step - loss: 0.0131 - accuracy: 0.9957 - val_loss: 0.1274 - val_a